In [1]:
%matplotlib inline
import anndata
from morphometrics.measure import measure_selected
import napari
import numpy as np
import pandas as pd
import pymeshfix
import pyvista as pv
import scanpy as sc
from skimage import draw
from skimage.measure import marching_cubes

rng = np.random.default_rng(42)

In [2]:
sphere_volume = (4/3) * np.pi  * (5**3)

cylinder_length = 15
cylinder_circle_area = sphere_volume / cylinder_length
cylinder_radius = np.sqrt(cylinder_circle_area / np.pi)
print(cylinder_radius)

3.3333333333333335


In [3]:
def make_cylinder():
    length_nominal = cylinder_length
    length_stdev = 1
    length = rng.normal(length_nominal, length_stdev)
    
    radius_nominal = cylinder_radius
    radius_stdev = 1
    radius = rng.normal(radius_nominal, radius_stdev)
    
    rr, cc = draw.disk((10, 10), radius)
    circle_image = np.zeros((20, 20), dtype=bool)
    circle_image[rr, cc] = True
    
    # extrude 
    
    return np.repeat(circle_image[np.newaxis, :, :], length, axis=0)


def make_sphere():
    radius_nominal = 5
    radius_stdev = 1
    radius = rng.normal(radius_nominal, radius_stdev)

    return draw.ellipsoid(radius, radius, radius)

In [4]:
object_functions = {
    'cylinder': make_cylinder,
    'sphere': make_sphere
}

In [5]:
label_image = np.zeros((300, 300, 300), dtype=int)
intensity_image = np.zeros((300, 300, 300), dtype=float)

label_index = 1
label_records = []

# embed the image
for x_start in np.arange(0, 300, 50):
    for y_start in np.arange(0, 300, 50):
        for z_start in np.arange(0, 300, 50):
            object_index = rng.choice(['cylinder', 'sphere'])
            object_func = object_functions[object_index]
            object_mask = object_func()
            
            
            object_coords = np.argwhere(object_mask)
            
            z_coords = z_start + object_coords[:, 0]
            y_coords = y_start + object_coords[:, 1]
            x_coords = x_start + object_coords[:, 2]
            
            label_image[z_coords, y_coords, x_coords] = label_index
            intensity_image[z_coords, y_coords, x_coords] = np.random.rand()
            
            label_records.append(
                {
                    'label_index': label_index,
                    'shape': object_index
                }
            )
            
            label_index += 1

label_table = pd.DataFrame(label_records)

In [6]:
viewer = napari.view_labels(label_image)
viewer.add_image(intensity_image)

<Image layer 'intensity_image' at 0x7f2d61390e20>

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/local0/kevin/miniconda3/lib/python3.8/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/local0/kevin/miniconda3/lib/python3.8/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/local0/kevin/miniconda3/lib/python3.8/asyncio/base_events.py", line 592, in run_until_complete
    self._check_running()
  File "/local0/kevin/miniconda3/lib/python3.8/asyncio/base_events.py", line 554, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/local0/kevin/miniconda3/lib/python3.8/site-packages/zmq/eventloop/zmqstream.py", line 452, in _handle_events
    self._ha

In [8]:
measurement_selection = [
    "surface_properties_from_labels",
    {
        "name": "regionprops",
        "choices": {
             "size": True,
            "intensity": True,
            "position": False,
            "moments": True
        },
    },
]

all_measurements = measure_selected(
    label_image=label_image,
    intensity_image=intensity_image,
    measurement_selection=measurement_selection,
)

all_measurements.head()

/local0/kevin/miniconda3/lib/python3.8/site-packages/skimage/measure/_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1101950558  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 15  Error-roundoff 1.3e-14  _one-merge 9.4e-14
  _near-inside 4.7e-13  Visible-distance 2.7e-14  U-max-coplanar 2.7e-14
  Width-outside 5.4e-14  _wide-facet 1.6e-13  _maxoutside 1.1e-13

precision problems (corrected unless 'Q0' or an error)
      4 degenerate hyperplanes recomputed with gaussian elimination
      4 nearly singular or axis-parallel hyperplanes
      4 zero divisors during back substitute
      4 zero divisors during gaussian elimination

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.



,surface_area,curvature_mean,curvature_stdev,curvature_0,curvature_10,curvature_20,curvature_30,curvature_40,curvature_50,curvature_60,...,moments_central-3-1-3,moments_central-3-2-0,moments_central-3-2-1,moments_central-3-2-2,moments_central-3-2-3,moments_central-3-3-0,moments_central-3-3-1,moments_central-3-3-2,moments_central-3-3-3,standard_deviation_intensity
label,,,,,,,,,,,,,,,,,,,,,
1,351.880820,-0.230401,0.159812,-1.361385,-0.414511,-0.315920,-0.262474,-0.207644,-0.170360,-0.155826,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.387779e-17
2,423.543536,-0.179639,0.038140,-0.288373,-0.235295,-0.206757,-0.188081,-0.179938,-0.173197,-0.166138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
3,292.330160,-0.246173,0.120118,-0.813107,-0.402705,-0.336488,-0.278129,-0.241226,-0.204010,-0.190953,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.220446e-16
4,312.991514,-0.240918,0.118863,-0.813107,-0.392184,-0.328405,-0.267096,-0.234484,-0.200221,-0.183031,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.110223e-16
5,410.412383,-0.187453,0.047633,-0.301419,-0.244006,-0.236497,-0.204346,-0.196645,-0.176200,-0.167652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.110223e-16


In [14]:
var = all_measurements.columns.to_frame()
X = all_measurements.fillna(0).to_numpy()
obs = pd.concat((all_measurements, label_table.set_index('label_index')), axis=1)['shape'].reset_index()


In [15]:
measurement_data = anndata.AnnData(X=X, var=var, obs=obs)

/local0/kevin/miniconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [16]:
print(measurement_data)

AnnData object with n_obs × n_vars = 216 × 214
    obs: 'index', 'shape'
    var: 0


In [17]:
sc.pp.scale(measurement_data)
sc.tl.pca(measurement_data, svd_solver='arpack')
sc.pl.pca(measurement_data, color=['shape'])

... storing 'shape' as categorical


In [18]:
sc.pl.pca_variance_ratio(measurement_data)

In [19]:
sc.pp.neighbors(measurement_data, n_pcs=6)
sc.tl.leiden(measurement_data)
sc.tl.umap(measurement_data)
sc.pl.umap(measurement_data, color='shape')

In [20]:
labels_properties = measurement_data.obs
labels_properties['umap_0'] = measurement_data.obsm['X_umap'][:, 0]
labels_properties['umap_1'] = measurement_data.obsm['X_umap'][:, 1]

viewer.layers['label_image'].properties = labels_properties

In [21]:
obs['shape_label'] = obs['shape'].map({'cylinder': 0, 'sphere': 1})
y = obs['shape_label'].to_numpy()

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [23]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [24]:
from sklearn import metrics

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [25]:
# display predictions

y_pred_all = clf.predict(X)

prediction_image = np.zeros_like(label_image)

for shape_prediction, label_index in zip(y_pred_all, obs['index']):
    prediction_image[label_image == label_index] = shape_prediction + 1

viewer.add_labels(prediction_image, color={1: [0, 1, 0, 1], 2: [1, 0, 1, 1]})

<Labels layer 'prediction_image' at 0x7f2cdcc3d640>